# Notebook 07: Morphological Topology

**Purpose**: Compute persistent homology to detect topological features (loops, voids) in strategy space.

**Workflow**:
1. Load SAE features (blocks 5, 20, 35)
2. Compute persistent homology using Ripser
3. Statistical validation via shuffled null distribution
4. Multi-resolution analysis (k=16, 32, 64, 128)
5. Cross-layer comparison
6. Visualization (barcodes, persistence diagrams)


## 1. Setup

In [ ]:
# ============================================================================
# COLAB SETUP - Run this cell first!
# ============================================================================
import sys
from pathlib import Path

# Detect if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Google Colab")

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Set up paths
    DRIVE_ROOT = Path('/content/drive/MyDrive/chaos')
    DRIVE_ROOT.mkdir(parents=True, exist_ok=True)

    print(f"Drive mounted. Project root: {DRIVE_ROOT}")

    # Install dependencies
    print("Installing dependencies...")
    !pip install -q torch>=2.0.0 h5py>=3.8.0
    !pip install -q matplotlib>=3.7.0 tqdm>=4.65.0
    !pip install -q scikit-learn>=1.2.0 scipy>=1.10.0
    !pip install -q ripser persim  # TDA libraries
    print("Dependencies installed!")

    # Unzip src.zip
    !unzip -n -q src.zip -d /content/
else:
    print("Running locally")
    DRIVE_ROOT = None

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import h5py
from contextlib import contextmanager

# ============================================================================
# TIMING INFRASTRUCTURE
# ============================================================================
@contextmanager
def timed_section(name):
    """Context manager to time code sections."""
    start = time.time()
    print(f"[START] {name}...")
    yield
    elapsed = time.time() - start
    print(f"[DONE] {name}: {elapsed:.1f}s ({elapsed/60:.1f} min)")

# Verify ripser installation
try:
    import ripser
    print(f"Ripser available")
except ImportError:
    print("ERROR: ripser not installed. Run: pip install ripser")

try:
    import persim
    print(f"Persim available")
except ImportError:
    print("WARNING: persim not installed. Wasserstein distances unavailable.")

# Add src to path
if IN_COLAB:
    sys.path.insert(0, '/content')
sys.path.insert(0, str(Path('.').absolute()))

# Import topology analysis module
from src.analysis.topology_analysis import (
    PersistenceResult,
    TopologyResult,
    compute_persistent_homology,
    shuffled_null_distribution,
    compute_significance,
    compute_persistence_stats,
    TopologyAnalyzer,
)

print("Modules loaded successfully")

# Configuration
CONFIG = {
    # Blocks to analyze
    'blocks': [5, 20, 35],  # Early, mid, late
    'primary_block': 35,    # Strategic layer (main analysis)
    
    # Matryoshka k-levels (if available)
    'k_levels': [16, 32, 64, 128],
    
    # Persistent homology settings
    'max_dim': 2,           # H0, H1, H2
    'n_subsample': 5000,    # Points for ripser (O(n^3) complexity)
    
    # Significance testing
    'n_shuffles': 100,      # Null distribution size
    'persistence_threshold': 0.1,  # Minimum persistence for significance
    
    # Paths
    'features_dir': 'outputs/data/sae_features',
    'output_dir': 'outputs/analysis/morphological_topology',
}

# ============================================================================
# COLAB: Configure paths for Drive storage
# ============================================================================
if IN_COLAB:
    CONFIG['features_dir'] = str(DRIVE_ROOT / 'data' / 'sae_features')
    CONFIG['output_dir'] = str(DRIVE_ROOT / 'analysis' / 'morphological_topology')

    print(f"Features dir: {CONFIG['features_dir']}")
    print(f"Output dir: {CONFIG['output_dir']}")

# Create output directories
Path(CONFIG['output_dir']).mkdir(parents=True, exist_ok=True)
(Path(CONFIG['output_dir']) / 'figures').mkdir(parents=True, exist_ok=True)
(Path(CONFIG['output_dir']) / 'persistence_diagrams').mkdir(parents=True, exist_ok=True)

## 2. Load Data

In [ ]:
# Load SAE features for primary block
def load_features(block_idx):
    """Load SAE features for a specific block."""
    features_path = Path(CONFIG['features_dir']) / f"block{block_idx}_features.h5"
    
    if not features_path.exists():
        print(f"Features not found: {features_path}")
        return None
    
    with h5py.File(features_path, 'r') as f:
        block_key = f'block{block_idx}'
        if block_key not in f:
            block_key = list(f.keys())[0]
        
        features = f[block_key][()].astype(np.float32)
    
    print(f"Loaded block {block_idx}: {features.shape}")
    return features

# Load primary block
features = load_features(CONFIG['primary_block'])

if features is not None:
    print(f"\nPrimary features: {features.shape}")
    print(f"Memory: {features.nbytes / 1e9:.2f} GB")

## 3. Primary Topology Analysis (Block 35)

In [ ]:
# Compute persistent homology with significance testing
topology_result = None

if features is not None:
    with timed_section("Primary topology analysis (block 35)"):
        # Initialize analyzer
        analyzer = TopologyAnalyzer(
            subsample_size=CONFIG['n_subsample'],
            max_dim=CONFIG['max_dim'],
            persistence_threshold=CONFIG['persistence_threshold'],
        )
        
        # Run full analysis with null distribution
        topology_result = analyzer.analyze(
            features,
            n_shuffles=CONFIG['n_shuffles'],
            seed=42,
        )

In [ ]:
# Display results
if topology_result is not None:
    print("="*60)
    print("TOPOLOGY ANALYSIS RESULTS")
    print("="*60)
    
    for dim in range(CONFIG['max_dim'] + 1):
        diagram = topology_result.observed.diagrams.get(dim, np.empty((0, 2)))
        stats = compute_persistence_stats(diagram)
        p_val = topology_result.p_values.get(dim, 1.0)
        
        sig_star = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        
        print(f"\nH{dim} ({['Components', 'Loops', 'Voids'][dim]}):")
        print(f"  Features: {stats['n_features']}")
        print(f"  Mean persistence: {stats['mean_persistence']:.4f}")
        print(f"  Max persistence: {stats['max_persistence']:.4f}")
        print(f"  Total persistence: {stats['total_persistence']:.4f}")
        print(f"  P-value: {p_val:.4f} {sig_star}")
    
    print("\n" + "="*60)
    
    # CRITICAL CHECK: H1 significance
    h1_p = topology_result.p_values.get(1, 1.0)
    if h1_p < 0.05:
        print(f"SIGNIFICANT H1 LOOPS DETECTED (p = {h1_p:.4f})")
    else:
        print(f"WARNING: H1 loops NOT significant (p = {h1_p:.4f})")
    print("="*60)

## 4. Visualization

In [ ]:
# Persistence diagram
if topology_result is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    colors = ['blue', 'red', 'green']
    titles = ['H0 (Components)', 'H1 (Loops)', 'H2 (Voids)']
    
    for dim in range(3):
        ax = axes[dim]
        diagram = topology_result.observed.diagrams.get(dim, np.empty((0, 2)))
        
        if len(diagram) > 0:
            # Plot points
            ax.scatter(diagram[:, 0], diagram[:, 1], 
                      c=colors[dim], alpha=0.6, s=30, label=f'n={len(diagram)}')
            
            # Diagonal line
            max_val = max(diagram.max(), 1)
            ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)
        
        p_val = topology_result.p_values.get(dim, 1.0)
        ax.set_xlabel('Birth', fontsize=12)
        ax.set_ylabel('Death', fontsize=12)
        ax.set_title(f'{titles[dim]}\np = {p_val:.4f}', fontsize=14)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.savefig(Path(CONFIG['output_dir']) / 'figures' / 'persistence_diagram.png',
                dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Persistence barcode (H1 only - most interesting)
if topology_result is not None:
    h1_diagram = topology_result.observed.diagrams.get(1, np.empty((0, 2)))
    
    if len(h1_diagram) > 0:
        # Sort by persistence (longest first)
        persistence = h1_diagram[:, 1] - h1_diagram[:, 0]
        sorted_idx = np.argsort(persistence)[::-1]
        
        # Show top 30
        n_show = min(30, len(h1_diagram))
        
        fig, ax = plt.subplots(figsize=(12, 8))
        
        for i, idx in enumerate(sorted_idx[:n_show]):
            birth, death = h1_diagram[idx]
            ax.barh(i, death - birth, left=birth, height=0.7, 
                   color='red', alpha=0.7, edgecolor='black')
        
        ax.axvline(x=CONFIG['persistence_threshold'], color='blue', 
                  linestyle='--', linewidth=2, label=f"Threshold: {CONFIG['persistence_threshold']}")
        
        ax.set_xlabel('Filtration Value (ε)', fontsize=12)
        ax.set_ylabel('H1 Feature Index (sorted by persistence)', fontsize=12)
        ax.set_title(f'H1 Persistence Barcode (Top {n_show} Loops)', fontsize=14)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3, axis='x')
        ax.invert_yaxis()
        
        plt.tight_layout()
        plt.savefig(Path(CONFIG['output_dir']) / 'figures' / 'persistence_barcode.png',
                    dpi=150, bbox_inches='tight')
        plt.show()
    else:
        print("No H1 features to visualize")

In [ ]:
# Null distribution comparison
if topology_result is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for dim, ax in enumerate(axes):
        dim_idx = dim + 1  # H1 and H2
        
        # Collect null counts
        null_counts = []
        for null_result in topology_result.null_distribution:
            null_diagram = null_result.diagrams.get(dim_idx, np.empty((0, 2)))
            null_persistence = null_diagram[:, 1] - null_diagram[:, 0]
            null_counts.append(np.sum(null_persistence > CONFIG['persistence_threshold']))
        
        null_counts = np.array(null_counts)
        
        # Observed count
        obs_diagram = topology_result.observed.diagrams.get(dim_idx, np.empty((0, 2)))
        obs_persistence = obs_diagram[:, 1] - obs_diagram[:, 0]
        obs_count = np.sum(obs_persistence > CONFIG['persistence_threshold'])
        
        # Plot
        ax.hist(null_counts, bins=20, edgecolor='black', alpha=0.7, 
               label=f'Null (mean={np.mean(null_counts):.1f})')
        ax.axvline(x=obs_count, color='red', linewidth=3, 
                  label=f'Observed: {obs_count}')
        
        p_val = topology_result.p_values.get(dim_idx, 1.0)
        ax.set_xlabel(f'H{dim_idx} Feature Count (persistence > {CONFIG["persistence_threshold"]})', fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.set_title(f'H{dim_idx} Null Distribution\np = {p_val:.4f}', fontsize=14)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(Path(CONFIG['output_dir']) / 'figures' / 'null_distribution_histogram.png',
                dpi=150, bbox_inches='tight')
    plt.show()

## 5. Cross-Layer Comparison

In [ ]:
# Compare topology across blocks (5, 20, 35)
block_results = {}

with timed_section("Cross-layer topology comparison"):
    for block_idx in CONFIG['blocks']:
        print(f"\n{'='*40}")
        print(f"Block {block_idx}")
        print(f"{'='*40}")
        
        block_features = load_features(block_idx)
        
        if block_features is not None:
            # Quick analysis (no null distribution for speed)
            result = compute_persistent_homology(
                block_features,
                max_dim=CONFIG['max_dim'],
                subsample=CONFIG['n_subsample'],
                seed=42,
            )
            
            block_results[block_idx] = result
        else:
            print(f"Skipping block {block_idx} (not found)")

In [ ]:
# Visualize cross-layer comparison
if len(block_results) > 1:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    block_names = {5: 'Early (5)', 20: 'Mid (20)', 35: 'Late (35)'}
    colors = {5: 'blue', 20: 'orange', 35: 'green'}
    
    for dim in range(3):
        ax = axes[dim]
        
        for block_idx, result in block_results.items():
            diagram = result.diagrams.get(dim, np.empty((0, 2)))
            
            if len(diagram) > 0:
                ax.scatter(diagram[:, 0], diagram[:, 1],
                          c=colors[block_idx], alpha=0.5, s=20,
                          label=f'{block_names[block_idx]} (n={len(diagram)})')
        
        # Diagonal
        ax.plot([0, 2], [0, 2], 'k--', alpha=0.3)
        
        ax.set_xlabel('Birth', fontsize=12)
        ax.set_ylabel('Death', fontsize=12)
        ax.set_title(f'H{dim} Cross-Layer Comparison', fontsize=14)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(Path(CONFIG['output_dir']) / 'figures' / 'cross_layer_comparison.png',
                dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Summary table
if len(block_results) > 0:
    summary_data = []
    
    for block_idx, result in block_results.items():
        for dim in range(CONFIG['max_dim'] + 1):
            stats = compute_persistence_stats(result.diagrams.get(dim, np.empty((0, 2))))
            summary_data.append({
                'Block': block_idx,
                'Dimension': f'H{dim}',
                'n_features': stats['n_features'],
                'mean_persistence': stats['mean_persistence'],
                'max_persistence': stats['max_persistence'],
                'total_persistence': stats['total_persistence'],
            })
    
    summary_df = pd.DataFrame(summary_data)
    print("\nCross-Layer Summary:")
    print(summary_df.to_string(index=False))

## 6. Wasserstein Distance Analysis (Optional)

In [ ]:
# Compute Wasserstein distances between layers
try:
    import persim
    
    if len(block_results) >= 2:
        print("\nWasserstein Distances (H1):")
        print("-" * 40)
        
        blocks = sorted(block_results.keys())
        
        for i in range(len(blocks)):
            for j in range(i + 1, len(blocks)):
                b1, b2 = blocks[i], blocks[j]
                
                dgm1 = block_results[b1].diagrams.get(1, np.empty((0, 2)))
                dgm2 = block_results[b2].diagrams.get(1, np.empty((0, 2)))
                
                if len(dgm1) > 0 and len(dgm2) > 0:
                    w_dist = persim.wasserstein(dgm1, dgm2)
                    print(f"  Block {b1} vs Block {b2}: {w_dist:.4f}")
                else:
                    print(f"  Block {b1} vs Block {b2}: N/A (empty diagrams)")
except ImportError:
    print("persim not installed. Skipping Wasserstein distances.")

## 7. Save Results

In [ ]:
if topology_result is not None:
    output_path = Path(CONFIG['output_dir'])
    
    # Save persistence diagrams
    for block_idx, result in block_results.items():
        np.savez(
            output_path / 'persistence_diagrams' / f'block_{block_idx}.npz',
            **{f'h{d}': result.diagrams.get(d, np.empty((0, 2))) 
               for d in range(CONFIG['max_dim'] + 1)},
            max_edge=result.max_edge,
            n_points=result.n_points,
            computation_time=result.computation_time,
        )
    
    # Save H1 statistics
    h1_stats = {
        'primary_block': CONFIG['primary_block'],
        'n_subsample': CONFIG['n_subsample'],
        'n_shuffles': CONFIG['n_shuffles'],
        'persistence_threshold': CONFIG['persistence_threshold'],
        'p_values': {str(k): float(v) for k, v in topology_result.p_values.items()},
        'observed_h1_count': len(topology_result.observed.diagrams.get(1, [])),
        'observed_h1_stats': compute_persistence_stats(
            topology_result.observed.diagrams.get(1, np.empty((0, 2)))
        ),
        'significant': topology_result.p_values.get(1, 1.0) < 0.05,
    }
    with open(output_path / 'h1_statistics.json', 'w') as f:
        json.dump(h1_stats, f, indent=2)
    
    # Save cross-layer comparison
    if len(block_results) > 1:
        summary_df.to_csv(output_path / 'cross_layer_summary.csv', index=False)
    
    # Save analysis summary
    summary = {
        'config': CONFIG,
        'h1_p_value': float(topology_result.p_values.get(1, 1.0)),
        'h1_significant': topology_result.p_values.get(1, 1.0) < 0.05,
        'blocks_analyzed': list(block_results.keys()),
    }
    with open(output_path / 'analysis_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"\nResults saved to {output_path}")
    print(f"  - persistence_diagrams/block_*.npz")
    print(f"  - h1_statistics.json")
    print(f"  - cross_layer_summary.csv")
    print(f"  - analysis_summary.json")
    print(f"  - figures/*.png")